## Load data

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras

Using Theano backend.


In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [3]:
datapath = '/media/sf_VBox_Shared/timeseries/KaggleEEG/train_1/prepared/'
resultpath = 'datapath'

In [4]:
ext = '.npy'
X_train = np.load(datapath+'X_train'+ext)
y_train_binary = np.load(datapath+'y_train_binary'+ext)
X_val = np.load(datapath+'X_val'+ext)
y_val_binary = np.load(datapath+'y_val_binary'+ext)
#X_test = np.load(datapath+'X_test'+ext)
#y_test_binary = np.load(datapath+'y_test_binary'+ext)

In [5]:
num_classes = y_train_binary.shape[1]
print(num_classes)

2


## Generate models

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam

In [7]:
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))

from mcfly import modelgen, find_architecture

In [8]:
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 10
                                  #model_type = 'CNN',
                                 )

In [9]:
for model, params, model_types in models:
    print(params)
    model.summary()

{'fc_hidden_nodes': 303, 'regularization_rate': 0.008797457773821373, 'filters': array([94, 90, 38, 31, 10, 19, 73]), 'learning_rate': 0.010806047643803933}
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
batchnormalization_1 (BatchNormaliz(None, 2400, 16)    32          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)    (None, 2400, 94)    4606        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormaliz(None, 2400, 94)    188         convolution1d_1[0][0]            
____________________________________________________________________________________________________
activation_1 (Activation)          

## Compare models

In [10]:
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                                 X_val, y_val_binary,
                                                                                 models,nr_epochs=5,
                                                                                  subset_size=500,
                                                                                  verbose=True, 
                                                                                  outputfile=resultpath+\
                                                                                  'modelcomparison.json')

Training model 0 CNN
Train on 100 samples, validate on 120 samples
Epoch 1/5
 80/100 [=======================>......] - ETA: 3s - loss: 96.3190 - acc: 0.5625

ValueError: Shape mismatch: x has 17520000 cols (and 20 rows) but y has 175200 rows (and 303 cols)
Apply node that caused the error: Dot22(Reshape{2}.0, dense_1_W)
Toposort index: 331
Inputs types: [TensorType(float32, matrix), TensorType(float32, matrix)]
Inputs shapes: [(20, 17520000), (175200, 303)]
Inputs strides: [(70080000, 4), (1212, 4)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{Composite{(i0 * (Abs((i1 + i2)) + i1 + i2))}}[(0, 1)](TensorConstant{(1, 1) of 0.5}, Dot22.0, InplaceDimShuffle{x,0}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [ ]:
for i in range(len(models)):
    name = str(models[i][1])
    find_architecture.plotTrainingProcess(histories[i], name)

In [ ]:
import pandas as pd
results = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
results

In [ ]:
results.to_csv(datapath+'results.csv')

## Train the best model for real

In [ ]:
best_model_index = np.argmax(val_accuracies)
#best_model_index = 3

In [ ]:
best_model, best_params, best_model_types = models[best_model_index]
print(best_model_index, best_model_types, best_params)

We make a copy of the model, to start training from fresh.

In [ ]:
best_model_copy = modelgen.generate_CNN_model(X_train.shape, num_classes, best_params['filters'], best_params['fc_hidden_nodes'],
                       best_params['learning_rate'], best_params['regularization_rate'])